## Flood-Health Vulnerability (FHV) Assessment

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer
import matplotlib.pyplot as plt
from functools import reduce
import fhv
pd.options.mode.chained_assignment = None

### Load datasets

In [116]:
# Load complete datasets
variable_name = pd.read_hdf('./data/variable_table.hdf')
variable = np.load('./data/variable.npy')
ncell = variable.shape[0]
# Load District ID
with rasterio.open(os.path.join('data', 'distid_peru.tif')) as src:
    did = src.read().squeeze()
    meta = src.meta.copy()
valid = did > 0
valid_id = did[did >0]
# Flood Inundation
with rasterio.open('./data/inundation_peru.tif') as src:
    fdep = src.read(1)
# Load population
# LandScan 2017: 30,555,036
# INEI 2017: 31,237,385
with rasterio.open('./data/population_peru.tif') as src:
    popu = src.read(1)
popu[(popu == -2147483647) | (popu < 0)] = 0    
popu = popu/popu.sum()*31237385                 
popu = popu.round()
npopu = popu.sum()                              # 31,177,745
# GDP(PPP) per grid (standard GDP * HYDE population)
# WorldBank's 2015 GDP(PPP): $369.342 billion (constant 2011 international $)
# WorldBank's 2015 GDP(PPP) per capita: $12,121 (constant 2011 international $)
with rasterio.open('./data/gdp_peru.tif') as src:
    gdp = src.read(1)
gdp[gdp == -9] = 0
gdp = gdp/gdp.sum()*369.342*10**9
# Hospitals and Travel-time to Health facilities
hospital = gpd.read_file('data/health_facility_MINSA_all.shp')
# - Flood inundation depth for hospitals
hospital = hospital[hospital.x != 0]
hospital['geometry'] = hospital['geometry'].to_crs(epsg=32717)
coords = [(x,y) for x,y in zip(hospital.geometry.x, hospital.geometry.y)]
with rasterio.open('./data/inundation_peru.tif') as src:
    hospital['fdep'] = [x[0] for x in src.sample(coords)]    # Inundation value
# - Travel-time to health facilities
ttime_nf = rasterio.open('./data/traveltime/health_noflood.tif').read(1)
ttime_yf = rasterio.open('./data/traveltime/health_yesflood.tif').read(1)
ttime_nf[np.isin(ttime_nf, [-32768, -1])] = 0
ttime_yf[np.isin(ttime_yf, [-32768, -1])] = 0
att = np.abs(ttime_yf - ttime_nf)

### FHV Assessment - Custom weight


In [32]:
# Custom weights
cwgt = pd.read_excel('./data/variable_weight_custom.xlsx')
cwgt = cwgt[['Name','Weight']] 
cwgt.drop(cwgt.tail(1).index,inplace=True)
cwgt = variable_name.merge(cwgt,on='Name').Weight.values
# Customized Vulnerability
vuln_custom = np.dot(variable,cwgt)
vuln_custom = fhv.ValidCellToMap(vuln_custom, valid, dtype='float32', nodata=-9999)
# Saving as GeoTiff
if True:
    fhv.GenerateRaster('./data/vulnability_custom.tif', meta, vuln_custom, 
                       new_dtype=rasterio.float32, new_nodata=-9999)

./data/vulnability_custom.tif is saved.


### FHV Assessment - Random weight

In [114]:
# Sensitivity analysis with random weights
nt = 1000      # Must be a multiple of 100
thsd = 0.55
freq = np.zeros([variable.shape[0],1])
for i in range(nt//100):
    rwgt = np.random.random([variable.shape[1],100])
    rwgt = rwgt/rwgt.sum(axis=0)
    score = np.dot(variable,rwgt)
    freq += (score > thsd).sum(1)[:,None]
vuln_random = freq/nt
vuln_random = fhv.ValidCellToMap(vuln_random.squeeze(), valid, dtype='float32', nodata=-9999)
# Temporarily
vuln_random[vuln_random < 0.7] = -9999
vulnRand = vuln_random.copy()
vulnRand[vulnRand>=0.7] = 1
vulnRand[vulnRand<0.7] = 0
vulnRand = vulnRand.astype('int8')
# Saving as GeoTiff
if True:
    fhv.GenerateRaster('./data/vulnability_random.tif', meta, vuln_random, 
                       new_dtype=rasterio.float32, new_nodata=-9999)
    fhv.GenerateRaster('./data/vulnability_random_logit.tif', meta, vulnRand, 
                       new_dtype=rasterio.int8, new_nodata=0)

./data/vulnability_random.tif is saved.
./data/vulnability_random_logit.tif is saved.


### Impact Assessment

In [120]:
# Impact Assessment
# (1) Affected popoulation
# *Affected population is assumed to increase linearly with water level until 2 meters.
thsd = 10
ratio = fdep.copy()
ratio[ratio <= thsd] = ratio[ratio <= thsd]/thsd
ratio[ratio > thsd] = 1
popuAfft = np.sum(ratio*popu)
# (2) Affected GDP
# *Affected GDP is assumed to increase linearly with water level from a damage of zero 
# for a water level of zero, to a maximum affected GDP at a water level of 3 meter.
thsd = 30
ratio = fdep.copy()
ratio[ratio <= thsd] = ratio[ratio <= thsd]/thsd
ratio[ratio > thsd] = 1
gdpAfft = np.sum(ratio*gdp)
# (3) Population in FHV zones
popuFHV = popu[vuln_custom >= 0.6].sum()
# (4) Populaion in PDV zone
popuPDV = popu[vuln_random >= 0.7].sum()
# (5) Affected Hospitals
# *If flood depth is over 1 meter
thsd = 10
nhsp = hospital.shape[0]
hspAfft = (hospital.fdep > 10).sum()
# Population with low accessibility
# *If Addtional Travel Time (ATT) is over 
thsd = 3*60
attAfft = popu[att > thsd].sum()
# Print the results of the Impacts Assessment
print('==================================================')
print('Impact Assessment')
print('--------------------------------------------------')
print('Affected population:\t{:>10,d} ({:.1f}%)'.format(int(popuAfft), popuAfft/npopu*100))
print('Affected GDP(PPP):\t${:>7,.0f}'.format(gdpAfft))
print('Population in FHV:\t{:>10,d} ({:.1f}%)'.format(int(popuFHV), popuFHV/npopu*100))
print('Area of FHV:\t{:.2f}%'.format((vuln_custom>=0.6).sum()/ncell*100))
print('Population in PVZ:\t{:>10,d} ({:.1f}%)'.format(int(popuPDV), popuPDV/npopu*100))
print('Area of PVZ:\t{:.2f}%'.format((vuln_random>=0.7).sum()/ncell*100))
print('Damaged Hospitals:\t{:6d}/{:3d} ({:.1f}%)'.format(int(hspAfft), nhsp, hspAfft/nhsp*100))
print('Low accessibility:\t{:>10,d} ({:.1f}%)'.format(int(attAfft), attAfft/npopu*100))
print('==================================================')

# print('PdomHzon area-A:\t{:.2f}%'.format(pdomHzonAreaA))
# print('PdomHzon area-B:\t{:.2f}%'.format(pdomHzonAreaB))
# print('Popu in pdomHzon-A:\t{:>10,d} ({:.1f}%)'.format(int(pdomHzonPopuA), pdomHzonPopuA/popuTotl*100))
# print('Popu in pdomHzon-B:\t{:>10,d} ({:.1f}%)'.format(int(pdomHzonPopuB), pdomHzonPopuB/popuTotl*100))
# print('Hzon in pdomHzon-A:\t{:.2f}%'.format(hzonInPdomA*100))
# print('Hzon in pdomHzon-B:\t{:.2f}%'.format(hzonInPdomB*100))
# print('Affected GDP:\t\t${:>7,.1f} B'.format(gdpAfft/10**9))
# print('Affected GDP per capita:${:>7,.1f} ({:.1f}%)'.format(gdpAfft/popuTotl, (gdpAfft/popuTotl)/(gdp.sum()/popuTotl)*100))
# print('*GDP(PPP) per capita:\t${:>7,d}'.format(int(gdp.sum()/popuTotl)))
# print('*Total population:     {:>,d}'.format(int(popuTotl)))
# print('--------------------------------------------------')
# print('Health Risk Assessment')
# print('--------------------------------------------------')
# print('Affected PHC:\t\t{:5d}/{:3d} ({:.1f}%)'.format(int(phcAfft), nphc, phcAfft/nphc*100))
# print('Affected Hospitals:\t{:6d}/{:3d} ({:.1f}%)'.format(int(hspAfft), nhsp, hspAfft/nhsp*100))
# print('ATT(>1hr) to PHC:\t{:>10,d} ({:.1f}%)'.format(int(popuAttPhc), popuAttPhc/popuTotl*100))
# print('ATT(>1hr) to Hospital:\t{:>10,d} ({:.1f}%)'.format(int(popuAttHsp), popuAttHsp/popuTotl*100))
# print('==================================================')

Impact Assessment
--------------------------------------------------
Affected population:	 1,047,864 (3.4%)
Affected GDP(PPP):	$1,428,493,508
Population in FHV:	   190,891 (0.6%)
Area of FHV:	1.90%
Population in PVZ:	   146,737 (0.5%)
Area of PVZ:	5.57%
Damaged Hospitals:	   250/8042 (3.1%)
Low accessibility:	   218,492 (0.7%)


0.01897337872694876

In [ ]:

# # Mapping
# import matplotlib.pyplot as plt
# from matplotlib.patches import Patch
# from matplotlib.colors import ListedColormap
# import matplotlib.colors as colors
# from rasterio.plot import plotting_extent
# # from pyproj import Proj, transform    # In case of re-projection
# import seaborn as sns
# sns.set(font_scale=1.5, style="white")
# # Adiministrative boundary
# peru_admin0 = gpd.read_file('./data/per_admbnda_adm0_2018.shp')
# peru_admin0 = peru_admin0.to_crs({'init':'epsg:32717'})
# lims = [-9086003.265, -7611331.177, -2103824.411, 31907.578]
# # Define the colors you want
# cmap = ListedColormap(["white", "tan", "springgreen", "darkgreen"])

# # Define a normalization from values -> colors
# norm = colors.BoundaryNorm([0.4, 0.5, 0.6, 1.0], 4)
# fig, ax = plt.subplots(figsize=(5, 10), facecolor='w')
# ax.set_axis_off()
# ax.set_aspect('equal')
# ax.axis(lims)
# chm_plot = ax.imshow(vuln_custom,cmap=cmap,norm=norm)
# peru_admin0.plot(ax=ax, color='white', edgecolor='black')

# # ax.set_title("Lidar Canopy Height Model (CHM)")
# # Add a legend for labels
# # legend_labels = {"tan": "short", "springgreen": "medium", "darkgreen": "tall"}
# # patches = [Patch(color=color, label=label)
# #            for color, label in legend_labels.items()]
# # ax.legend(handles=patches,
# #           bbox_to_anchor=(1.35, 1),
# #           facecolor="white")
# plt.tight_layout()
# plt.show()
